# Exploration by Random Network Distillation

**Abstract**
<br>
exploration bonus 를 활용하여 1. easy to implement, 2. minimal overhead to computation 한 딥러닝 기법을 소개합니다.
> The bonus is error of a neural network predicting features of the observations given by a fixed randomly initializd neural network.
> flexibly combine intrinsic and extrinsic rewards
<br>

우리는 random network distillation (RND) bonus with increased flexibility 을 통하여 exploration 이 많이 필요한 몇 몇 atari 게임들에 있어 의미있는 진전을 이루었습니다. 특히 심층 강화학습 기법들에서 가장 어렵다고 알려진 Montezuma's Revenge 라는 게임에 있어 최고의 성능을 보였으며, 어떠한 데모를 통한 학습이나 상태의 접근 없이 전문 게임 플레이어 보다 나은 점수를 얻은 첫 번째 기법입니다.


## 1. Introduction
강화학습 기법들은 expected return of policy 를 최대화 하는 방법으로 동작합니다. 그리고 이러한 방법은 환경이 dense rewards 를 가질 수 있을 때에는 잘 동작하지만, sparse 하거나 찾기 힘들 때에는 실패하기 쉽습니다. 현실 세계에서 강화학습 에이전트가 문제를 해결하기에 dense reward functions 을 매번 만들어 내는 것은 실용적이지 않을 것입니다. 이러한 상황에서 환경을 탐험하는 직접적인 방법이 필요합니다.
![Figure.1](images/figure1.png)

 최근 강화학습 가장 도전적인 일들을 해결하는 데에 추천되고 있으며, 병렬로 실행되는 환경에서 대용량 데이터의 처리를 필요로 합니다. 이러한 점을 비추어 보았을 때에 대용량 데이터 처리가 가능한 확장성 있는 탐험 기법들이 필요로 하는데, 최근에 소개된 많은 기법(based on counts, pseudo-counts, information gain or prediction gain)들은 병렬 환경의 대용량 처리에는 확장성이 떨어집니다.

 본 논문에서 소개하는 단순하면서도 구현하기 쉬운 탐험 보너스 기법은 고차원 observations 에서도 잘 동작하며, policy optimization 알고리즘도 적용할 수 있으며, 배치 처리에서 단지 single forward pass of neural network 만을 필요로 하므로 아주 효율적입니다.
  우리의 탐사 보너스는 신경망이 훈련 된 것과 유사한 예에서 예측 오차가 현저하게 낮은 경향이 있다는 관찰에 기반하며, 이러한 사실은  에이전트가 과거에 경험을 학습한 네트워크가 새로운 경험의 참신함을 계량화 하는 데에 예측 오류를 활용할 수 있게 되었다.
  
  많은 저자들이 언급하였지만 예측오류를 최대화 하는 에이전트들은 입력 데이터들에 대한 확률적 함수와 같이 transitions 하기 쉽게 되는데, 예를 들어 현재 observation과 에이전트의 action 에 대한 다음 observation 을 에측하는 문제가 있다고 한다면, 에이전트는 예측 오류를 줄이기 위해 stochastic transitions 을 찾으려고 할 것이고 일정하고 랜덤하게 변화하는 noise TV 와 같은 혹은 coin tosses 등에 쉽게 휘말리게 된다. 이러한 observation 들을 확인함으로써, 예측의 오류가 아니라 상대적인 개선을 정량화 할 수 있는 기법에 대한 사용하는 데에 동기부여를 할 수가 있었다. 하지만 이러한 기법들은 효과적인 구현이 아주 어렵다.
  
 위에서 언급한 정답이 입력에 대한 deterministic 함수인 문제를 예측하는 데에 탐사 보너스를 정의함으로써 발생하는 의도하지 않은 확률론적인 것에 대한 대안을 제시합니다.  랜덤 초기화된 고정된 신경망에 현재의 observation 을 입력으로 한 output 을 예측하는 방법이다.
 
 <중략>
 
 extrinsic rewards 를 같이 사용하지 않더라도 RND 탐사 보너스만으로 몬테주마 복수 게임의 절반 이상의 방을 찾을 수 있었다. 탐사 보너스와 extrinsic rewards 를 통합하기 위해서 extrinsic, intrinsic rewards 의 2가지 reward streams 를 적용한 PPO 알고리즘을 소개하였습니다. 이러한 방법으로 episodic and non-episodic returns 를 통합할 수 있었꼬, 2가지 rewards 에 대하여 별도의 discount rates 를 적용할 수가 있었습니다.
> episodic 은 해당 에피소드 당 얻는 extrinsic 관련 reward 이고 non-episodic 은 현재 에피소드 별 상황과는 무관하게, random network 와 predict network 를 통해 observation 예측에 따른 error 를 reward 를 말한다.
 

## 2. Method

### 2.1 Exploration bonuses
환경 보상이 너무 sparse 한 경우 에이전트에게 탐험을 하도록 북돋우는 종류 중에 하나이다.

#### Encourage visit novel-state w/ $i_t$
Count-based exploration - 자주 방문하는 곳 보다 novel state 방문 보상이 더 커야 한다
##### 1. Tabular case
> $i_t = 1/{n_t(s)}\ and\ i_t = 1/\sqrt{n_t(s)} \ - \ n_t(s)$ : 상태 s 의 방문 횟수를 활용

##### 2. Non-tabular case
> 모든 state 를 방문하기 어렵기 때문에 상태의 밀도 추정의 변화를 pseudo-count 로 활용 할 수 있는데 이전에 방문한 적이 없더라도 유사한 곳은 긍정정인 보상을 반환하는 방식이다

#### Use $i_t$ for predicting error while agent transition
##### 1. Generic prediction problem w/ forward and inverse dynamics
##### 2. Non-generic prediction problem
> 현재의 에이전트 물리적 속성(위치 등)과 예측된 것과의 오류를 줄이는 방향으로 학습
<br>

### 2.2 Random Network Distillation
이 논문에서는 prediction problem 이 드물게 생성되는 다른 접근을 하겠다

#### 2개의 신경망을 사용
##### 1. 랜덤 초기화된 고정 _target network_ : 해결해야 하는 예측 문제
##### 2. 에이전트의 수집된 데이터에 의해 학습된 _predictor netwrok_ : 랜덤 네트워크와의 예측 오류를 최소화하도록 학습
> 1. MSE 값을 최소화 하도록 GD 를 통하여 paramters $\theta_{\hat f}$ 를 학습합니다
2. 이러한 과정은 랜덤 초기화된 신경망을 학습된 네트워크로 만들어 냅니다
3. 여기서 예측 오류는 예측기의 입장에서 여태까지 학습해 왔던 것(기존 정답이라고 생각했던 반복적인 행동) 과는 다른 행동을 할 때에 높게 나타나게 됩니다 (왜냐하면 random network 가 target 이므로)
<br>

#### 직관적인 확인을 위해 MNIST 적용
##### 1. 랜덤 초기화된 target network 를 따라하는 predictor 신경망을 훈련시킵니다
##### 2. 새로운 예제를 통해 predictor network 테스트를 하고 MSE를 측정합니다
> 1. 숫자 이미지(States)를 통해 번호를 구분(Actions)하는 행위를 예측하는 강화학습입니다
2. 기존에 많이 학습했던 0번 이미지는 자주 방문한 상태이고,
3. 기타 숫자의 이미지들은 자주 방문하지 않은 상태라고 말할 수 있습니다

![novelty detection](images/figure2.png)
> 현재의 MSE 값은 novel state 를 인지하는 좋은 함수임을 확인할 수 있습니다.
1. 좌측 상단의 오류가 큰 경우 학습 샘플링에 해당 숫자 이미지 상태가 거의 발생하지 않은 경우이며
2. 우측 하단의 오류가 작은 경우 충분히 해당 상태가 발생한 경우입니다
3. 과도하게 최적화된 예측자가 완전히 랜덤하게 되어버리는 문제가 있을 수도 있을텐데, 실험에서와 같이 표준 그래디언트 방식은 그정도로 과도하게 생성되지는 않는 다는 것을 알수 있습니다.

#### 2.2.1 Sources of prediction errors
> 1. 학습 데이터가 부족할 때 - Lack of training data : 이 요소로 인해 예측 오류를 탐사 보너스로 사용할 수 있습니다
2. 예측 대상 모델이 확률적일 때 - Stochasticity of target model : 이 요소가 noisy-TV 문제를 일으킬 수 있습니다
3. 학습 모델을 잘 못 사용할 때 - Misspecification of model : 대상함수의 복잡도에 비해 모델이 너무 단순한 경우
4. 모델 학습에 실패했을 때 - Failure of learning dynamics
##### **RND 의 경우 target network 가 deterministic 하다는 점과, predictor network 와 같은 model-class 를 가짐으로써 2, 3번 요소가 문제시 되지 않습니다**

#### 2.2.2 Relation to uncertainty quantification
> 1. 왜 norm 의 합과 차가 같다고 말하는 지 이유를 잘 모르겠습니다
![MSE](images/mse.png)
![MSE](images/argmin.png)
##### **distillation error 는 constant zero function 을 예측하는 quantification of uncertainty 와 같다고 볼 수 있습니다**


### 2.3 Combining intrinsic and extrinsic returns
#### Use intrinsic rewards only
> 1. 더 나은 탐험을 위해 non-episodic 문제를 다룰 때에는 intrinsic rewards 만 씁니다
2. 아주 난이도 있는 게임을 깨기 위해 처음부터 계속 지루한 반복을 하는 것을 피할 수 있습니다
3. 다만 게임의 시작 부분에 가까운 보상을 노리고 고의로 게임을 다시 시작하는 경우가 있을 수도 있습니다
#### Return is linear in the rewards
> 1. 내적, 외적 보상을 적절히 합하는 것은 분명하지 않은 방법입니다
2. 반환 값은 보상에 선형적인 특징을 가지고 있으므로 최종 보상은 두 값을 더합니다
3. 내적 보상은 non-stationary 하고, 외적 보상은 stationary 한 특징을 가집니다
4. 그래서 외적 내적 보상을 별도로 계산하여 넣을 수 있고 각가의 discount factor 적용하는 점도 중요합니다

### 2.4 Reward and observation normalization
> 1. 예측 오류를 탐험 보너스로 사용할 때에 환경에 따라 값의 편차가 큰 점이 문제입니다
2. Intrinsic rewards 값의 표준편차 값으로 보상을 정규화 하는 방법을 사용합니다
3. Observation normalization 은 딥러닝에서 아주 중요한 역할을 합니다. 특히 random neural network 를 target 으로 사용하는 경우 paramter 가 frozen 되어 있으므로 sacle 이 다른 dataset 의 경우 수렴하지 않을 수 있기 때문입니다.
4. value 값에서 mean 을 빼고 standard deviation 으로 나누어진 수를 -5 ~ 5 사이의 값으로 clip 하여 사용합니다
5. policy network 를 제외한 나머지 predictor, target network 는 공히 normalization 을 하여 사용합니다

## 3. Experiments

### 3.1 Pure exploration
### 3.2 Combining episodic and non-episodic returns
### 3.3 Discount factors
### 3.4 Scaling up training
### 3.5 Recurrence
### 3.6 Comparison to baselines
### 3.7 Qualitative analysis: Dancing with skulls

## 4. Related works

## 5. Discussion

## .

## Q&A
### 1. state density estimation?
### 1. forwrad dynamics?